## How to train the whole flow

---

This notebook shows how you can train your simple model from the .pickle file. 

In [1]:
import sys
sys.path.append("../") 
# go to parent dir, a bit hacky but avoid having to put all the notebooks in root directory 😬

In [11]:
import time
import pandas as pd
from sklearn import preprocessing
from iml_group_proj.features.simple import get_simple_features
from iml_group_proj.features.bert import get_BERT_features

from iml_group_proj.evaluation import evaluate_many
from iml_group_proj.train_models import train_models
from iml_group_proj.features.common.data import load
from iml_group_proj.models.mlp import BASIC_MLP

In [3]:
IS_BOW = False # Bag-of-words or TfIdf

In [4]:
classes, train, test = load("../trainTest.pk")

train.head()

,class,title,sypnosis,id
12644,T,"Basic highway surveying, course HT.109 course ...",Provides an overview of basic surveying defini...,63915520
4460,P,We played marbles /,When friends playing on a Civil War battlefiel...,37269285
7596,E,Ancient monuments of the Mississippi Valley /,Complete 48 plate collection from the 1848 ed....,26965717
1004,Q,Human body revealed /,With in-depth information about each part of t...,50759512
23734,E,Ancient America /,Maps and text offer information on the culture...,21197826


### Define a list of models to train

In [ ]:
# Training Phase, 
models = [
        (BASIC_MLP, None, 'MLP_100'),
        # (GaussianNB(), {},'NaiveBayes'),
        # (SVC(), {"gamma": ["auto"], "kernel": ["rbf", "sigmoid", "poly"]},'SVM'),
        # (SVC, {"gamma": "auto", "kernel": "rbf"},'SVM'),
        ]

### Extracting features from text

In [6]:
train = train.sample(frac=0.5) # just to make the code run faster

##### TF-IDF

return a sparse matrix of dimension [records, words]. 
Either count or tf-idf weighted. (not standardized)

In [7]:
%%timeit
X_train, X_test = get_simple_features(train, test, bow=IS_BOW, only_title=False)
X_train = X_train.toarray() 
X_test = X_test.toarray()
# (Could have use .toarray() inside the get_simple but in case want to do something with the sparse matrix.)

1.23 s ± 127 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### BERT Embeddings

return a list of averaged embeddings for each record

In [ ]:
%%timeit
X_train, X_test = get_BERT_features(train, test)

In [ ]:
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(train["class"])
y_test = le.transform(test["class"])

### Training Phase

In [ ]:
trained_models = train_models(models, X_train, y_train)
result_df = evaluate_many(trained_models, X_train, y_train, X_test, y_test)

In [ ]:
result_df.head(10)